# Image Processing: Histogram of Oriented Gradients (HOG) with MNIST Dataset

In [5]:
import torch
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms
from skimage.transform import resize
from skimage.feature import hog
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# get training and test set
mnist_train = MNIST("./data/", download = True)
mnist_test = MNIST("./data/", train = False, download = True)